In [1]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 23.1 MB/s eta 0:00:00


In [2]:
!python3 -m nltk.downloader wordnet

/opt/conda/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Libraries to drop english words and tokenize the text
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

import contractions
import string

# Libraries for TF-IDF model and Logistic Regression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-insincere-questions-classification/sample_submission.csv
/kaggle/input/quora-insincere-questions-classification/embeddings.zip
/kaggle/input/quora-insincere-questions-classification/train.csv
/kaggle/input/quora-insincere-questions-classification/test.csv


In [5]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

---
# <font color=green>Preprocess the data</font>

In [6]:
# DataFrame with Test data
test_df = pd.read_csv(f'/kaggle/input/quora-insincere-questions-classification/test.csv')
# DataFrame with Train Data
train_df = pd.read_csv(f'/kaggle/input/quora-insincere-questions-classification/train.csv')

In [7]:
#  The Function for text preprocessing
def tokenize_string(text):

    # Before lemmatizing replace all constructions with normal words
    text_upd = contractions.fix(text)

    # Tokenize the data and use only lower letters
    words = word_tokenize(text_upd.lower())
    
    # Create a lemmatizer object
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(word, pos = "v") for word in words] 
    
    # # Get rid of punctuation
    words = [word for word in lemmas if word not in string.punctuation]
    
    # Remove stop words
    # Stop words corpus (179 in total)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    return words

In [8]:
# Tokenizing the series for test data
print('I have just started updatind test_df')
test_df['Preprocessed_text'] = test_df.question_text.apply(tokenize_string)

# Tokenizing the series for train data
print('I have just started updatind train_df')
train_df['Preprocessed_text'] = train_df.question_text.apply(tokenize_string)

I have just started updatind test_df
I have just started updatind train_df


In [9]:
train_df.head()

,qid,question_text,target,Preprocessed_text
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0,"[quebec, nationalists, see, province, nation, ..."
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0,"[adopt, dog, would, encourage, people, adopt, ..."
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0,"[velocity, affect, time, velocity, affect, spa..."
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0,"[otto, von, guericke, use, magdeburg, hemisphe..."
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0,"[convert, montra, helicon, mountain, bike, cha..."


---
# <font color=green>Creating baseline model based on Logistic Regression</font>

In [10]:
train_df['text'] = train_df.Preprocessed_text.apply(lambda x: ' '.join(x))
test_df['text'] = test_df.Preprocessed_text.apply(lambda x: ' '.join(x))

In [11]:
# Create the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

In [12]:
# Vectorize the sentences
X_train = vectorizer.fit_transform(train_df['text'])
X_test = vectorizer.transform(test_df['text'])

y_train = train_df.target.values

In [13]:
# Train a logistic regression model on the vectorized data
clf = LogisticRegression(random_state=0, solver='liblinear').fit(X_train, y_train)

In [14]:
# Make a prediction on new data
y_pred = clf.predict(X_test)

---
# <font color=green>Making the final file</font>

In [15]:
pd.DataFrame({'qid': test_df.qid, 'prediction': y_pred}).to_csv('submission_logistic_reg.csv', index='qid')